In [2]:
import pandas as pd
import numpy as np
from collections import Counter

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier

from nltk.corpus import stopwords
import string
import re
from bs4 import BeautifulSoup
import string
from unicodedata import normalize as norm

In [3]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [4]:
DATASET_COLUMNS = ["target", "ids", "date", "flag", "user", "text"]
DATASET_ENCODING = "ISO-8859-1"
TRAIN_SIZE = 0.8

In [7]:
df = pd.read_csv('../data/portuguese-tweets-for-sentiment-analysis/NoThemeTweets.csv')

In [8]:
df.head()

,id,tweet_text,tweet_date,sentiment,query_used
0,1031761728445530112,@Tixaa23 14 para eu ir :),Tue Aug 21 04:35:39 +0000 2018,Positivo,:)
1,1031761040462278656,@drexalvarez O meu like eu já dei na época :),Tue Aug 21 04:32:55 +0000 2018,Positivo,:)
2,1031760962372689920,Eu só queria conseguir comer alguma coisa pra ...,Tue Aug 21 04:32:37 +0000 2018,Positivo,:)
3,1031760948250456066,:D que lindo dia !,Tue Aug 21 04:32:33 +0000 2018,Positivo,:)
4,1031760895985246208,"@Primo_Resmungao Pq da pr jeito!!é uma ""oferta...",Tue Aug 21 04:32:21 +0000 2018,Positivo,:)


In [9]:
def cleaning_text(text):
    
    text = text.lower()
    
    text = re.sub('(\\n)+', ' ', text)
    
    text = ' '.join([word for word in text.split() if word not in stopwords.words('portuguese') and word not in string.punctuation])
    
    text = norm('NFKD', text).encode('ascii', 'ignore').decode()
    
    text = re.sub('\@\S*', '', text)
    
#     text = ''.join([char for char in text if char not in string.punctuation])
    
    return text

In [11]:
Counter(df['sentiment'])
# 0 - Negative
# 4 - Positive

Counter({'Positivo': 263107, 'Negativo': 522707})

In [13]:
df['text_clean'] = df['tweet_text'].apply(cleaning_text)

In [14]:
df.to_csv("portuguese_data.csv")